In [1]:
from potassium import Potassium, Request, Response
from transformers import pipeline
import torch
import time

app = Potassium("my_app")

# @app.init runs at startup, and initializes the app's context
@app.init
def init():
    # Check if there is a GPU
    device = 0 if torch.cuda.is_available() else -1

    # Defind the model, using the "pipeline" interface from transformers
    bertModel = pipeline('fill-mask', model='bert-base-uncased', device=device)
   
    
    context = {
        "model_b": bertModel,
    }

    return context



# @app.handler is an http post handler running for every call
@app.handler("/bert")
def handler(context: dict, request: Request) -> Response:
    
    # our model is from 'model_b' as defined in the context earlier
    model = context.get("model_b")

    # our prompt is from the 'prompt' key in the user request
    prompt = request.json.get("prompt")

    # the outputs is created by passing the user request to the model
    outputs = model(prompt)

    # returned
    return Response(
        json = {"outputs": outputs}, 
        status=200
    )

if __name__ == "__main__":
    app.serve()

------
Starting Potassium Server 🍌
Running init()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Started 1 workers
Serving at http://0.0.0.0:8000
------
